In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## these were dates when data was collected

30 zip codes were considered

In [3]:
dates = ['2019-01-25', '2019-01-27', '2019-01-30', '2019-01-31', '2019-02-02', '20129-02-03', '2019-02-09', '2019-02-17', '2019-03-02']

### for now , we only focus on data from seattle for a single date in a single zipcode (Seattle). This is more of a prototype of how the analysis might go

### as you can see below, the sentiment analysis is not yet very predictive, owing in large part to a dearth of positive exampels for adopted pets
As of now TODO
- find a better way to clean data
- use longitudinal aspect of the data (i.e. pet records disappearing with time) as additional cue for adoption
- if we still find positive data to be a defecit, we need to combine all positive records and sample from negative record distribution. 
- try other modeling methods.
- use better word tokenizer 
- take the "ADOPTED!" out of positive records. As soon as this hting starts working, it should obviously be picking up on that as by far the most predictive token

In [4]:
import json


In [5]:
finname = 'data/seattle/2019-01-25bigdata.json'

In [10]:
data = ( json.load(open(finname,"r")) )

In [14]:
data_str =  json.dumps(data)

In [16]:
len(data_str)

4143468

In [17]:
data_str_split = data_str.split(",")


In [18]:
len(data_str_split)

82251

### lets take a look at the data

In [26]:
data_str_split[0:200]

['{"@encoding": "iso-8859-1"',
 ' "@version": "1.0"',
 ' "petfinder": {"@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance"',
 ' "lastOffset": {"$t": "1000"}',
 ' "pets": {"pet": [{"options": {"option": {"$t": "housetrained"}}',
 ' "status": {"$t": "A"}',
 ' "contact": {"phone": {"$t": "7133045266"}',
 ' "state": {"$t": "WA"}',
 ' "address2": {}',
 ' "email": {"$t": "rhondabarron@earthlink.net"}',
 ' "city": {"$t": "Seattle"}',
 ' "zip": {"$t": "98106"}',
 ' "fax": {}',
 ' "address1": {}}',
 ' "age": {"$t": "Young"}',
 ' "size": {"$t": "M"}',
 ' "media": {"photos": {"photo": [{"@size": "pnt"',
 ' "$t": "http://photos.petfinder.com/photos/pets/40221309/1/?bust=1513309460&width=60&-pnt.jpg"',
 ' "@id": "1"}',
 ' {"@size": "fpm"',
 ' "$t": "http://photos.petfinder.com/photos/pets/40221309/1/?bust=1513309460&width=95&-fpm.jpg"',
 ' "@id": "1"}',
 ' {"@size": "x"',
 ' "$t": "http://photos.petfinder.com/photos/pets/40221309/1/?bust=1513309460&width=500&-x.jpg"',
 ' "@id": "1"}',
 ' {"@s

### these permutations seem to be reliable

In [33]:
for line in data_str_split:
    if 'Adopted' in line:
        print(line)

 "name": {"$t": "Kasey - Adopted!"}
 "name": {"$t": "Coco - Adopted"}


In [37]:
for line in data_str_split:
    if 'ADOPTED' in line:
        print(line)

 "description": {"$t": "ADOPTED\n\nCoco is an adorable female mixed breed dog
 "name": {"$t": "Gizmo WAS ADOPTED!!"}


### this permutation is not reliable

In [38]:
for line in data_str_split:
    if 'adopted' in line:
        print(line)

 watching others being adopted. This made me very sad and depressed. Thanks to 6dogrees Rescue  I am not only out of the shelter
 we \u00e2\u0080\u009cSave it forward\u00e2\u0080\u009d; so each time a dog is adopted we are allowed to save another. Adoption fees are non-negotiable and are in place to allow thorough care for the dogs. \n\n6dogrees adoptions are limited to residents of Washington State.\n"}
 we \u00e2\u0080\u009cSave it forward\u00e2\u0080\u009d; so each time a dog is adopted we are allowed to save another. Adoption fees are non-negotiable and are in place to allow thorough care for the dogs. 6dogrees adoptions are limited to residents of Washington State.\nApply online at www.6dogrees.com"}
 we \u00e2\u0080\u009cSave it forward\u00e2\u0080\u009d; so each time a dog is adopted we are allowed to save another. Adoption fees are non-negotiable and are in place to allow thorough care for the dogs. \n\n6dogrees adoptions are limited to residents of Washington State.\n\n"}
 we 

In [78]:
all_descriptions = []
descrip_count = 0
adopted_records = []

for line in data_str_split:
    if (('ADOPTED' in line) or ('Adopted' in line)):
        adopted_records.append(descrip_count)
    if '"description"' in line:
        all_descriptions.append(line)
        descrip_count+=1
    

In [79]:
adopted_records

[461, 494, 494, 770]

### this will be very hard with only 4 positive examples

In [80]:
len(all_descriptions)

1000

In [29]:
len(all_descriptions)

1000

In [39]:
all_descriptions[0:10]

[' "description": {"$t": "Meet Blitzen\\n\\nBlitzen is a handsome male retriever mix.. He is a Fun and loving boy who loves to play with other dogs. He is crate trained and house trained. Blitzen and would love to have a friend to play with! Blitzen is shy around new humans unless he is with a dog friend.  Blitzen would have to go to a home with another doggie to show him the ropes!   He is the perfect play buddy for any other dog. \\n\\nHe has received all his vaccinations',
 ' "description": {"$t": "Mia (formerly know as \\"Momma Mia\\") has a long story!  She was a stray on the streets and had puppies under a house.  A kind lady at the coffee shop fed her thru her pregnancy. In summer 2015 she had puppies and they were taken in my a rescue group and finally Mia was caught by 3 ladies and a policeman who also helped. She went to a boarding facility for socialization and came to live with her foster in November of 2015 where she still resides!\\nHer foster has 12 other dogs and if she

### clearly flawed because some descriptions span multiple lines. Will need to deal with this in future version

In [40]:
import nltk


In [62]:
alldescriptions_collapsed = ''
for line in all_descriptions:
    alldescriptions_collapsed = alldescriptions_collapsed+line+'\n PET END \n'

In [63]:
len(alldescriptions_collapsed)

237923

In [64]:
pet_tokens = nltk.word_tokenize(alldescriptions_collapsed)


In [65]:
len(pet_tokens)

52965

In [66]:
len(set(pet_tokens))

4294

In [67]:
set(pet_tokens)

{'mild',
 'Milo',
 'dwellers',
 'thru',
 'milkshake',
 'album_id=1757064957645627\\n\\nMeet',
 'profile.\\n\\nThe',
 'Update',
 'call',
 'spend',
 'details.\\n\\nMeet',
 'Koda',
 'terms',
 'hike',
 'belly',
 'dedicated',
 'my\\nforever',
 'Dairy',
 'brown',
 'settle',
 'Kid',
 'lbs.\\n\\nand',
 'truly',
 'Dogo',
 'album_id=2272915169393934\\n\\nMeet',
 'bell',
 '[',
 'strolls',
 'littermates',
 'Moe',
 'Jan',
 'Grove',
 'Susu',
 'man',
 'knows',
 'Giada',
 'experienced',
 'Costco',
 'match.\\n\\nHermione',
 '20',
 'bachelorette',
 'smells',
 'smarty',
 'Queenie',
 'Argentino',
 'sooo',
 'type=3\\n\\nWe',
 'THIS',
 'fur',
 'growly',
 'BB',
 'laughing',
 'Tiana',
 '10',
 'impact',
 'couch',
 'Zena',
 '\\nOur',
 'DDB/Bloodhound',
 'clean',
 'committed.\\n\\nAllie',
 'male\\nWeight',
 'volunteers',
 'younger',
 'send',
 'shy',
 'Gabi',
 'Samaritan',
 'NORMAN',
 'Girl.\\n\\nI',
 '5-6-month-old',
 'case',
 'brain',
 'him',
 'or',
 'beagle',
 '-Contact',
 'himself',
 'signs',
 'Angelina',
 'w

In [70]:
from nltk.book import FreqDist

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [68]:
text1 = nltk.Text(pet_tokens)

In [71]:
dist = FreqDist(text1)


In [72]:
dist

FreqDist({'``': 2974, ':': 2303, "''": 2041, '.': 1561, 'a': 1556, 'and': 1387, 'is': 1353, '$': 1026, 'description': 1001, '{': 1000, ...})

In [73]:
sorted(dist.items(), key=lambda item: item[1])[::-1][0:20]

[('``', 2974),
 (':', 2303),
 ("''", 2041),
 ('.', 1561),
 ('a', 1556),
 ('and', 1387),
 ('is', 1353),
 ('$', 1026),
 ('description', 1001),
 ('END', 1000),
 ('PET', 1000),
 ('{', 1000),
 ('t', 987),
 ('to', 836),
 ('!', 558),
 ('with', 464),
 ('I', 445),
 ('in', 422),
 ('old', 405),
 ('the', 375)]

### list of words with length 5,  and  occuring 30 times in the descriptions

In [75]:
sorted([w for w in dist if len(w) > 5 and dist[w] > 30])


['Adoption',
 'Animal',
 'California',
 'Chihuahua',
 'Female',
 'PLEASE',
 'Petfinder',
 'Please',
 'Profile',
 'Seattle',
 'Shepherd',
 'Terrier',
 'active',
 'adoption',
 'adorable',
 'application',
 'around',
 'beautiful',
 'before',
 'bigdogrescueproject.org/adopt',
 'description',
 'directly',
 'excited',
 'family',
 'female',
 'forever',
 'foster',
 'friendly',
 'handsome',
 'high-kill',
 'inquire',
 'little',
 'looking',
 'loving',
 'months',
 'neutered',
 'official',
 'online',
 'people',
 'playful',
 'pounds',
 'rescue',
 'rescued',
 'shelter',
 'through',
 'trained',
 'training',
 'transported',
 'wanted',
 'weighs',
 'wonderful']

### words consiting of alphabetical letters that occur in 

In [76]:
sorted([w for w in dist if w.isalpha() and dist[w] > 100])

['END',
 'He',
 'I',
 'Meet',
 'My',
 'PET',
 'She',
 'a',
 'about',
 'adoption',
 'am',
 'an',
 'and',
 'application',
 'are',
 'as',
 'at',
 'be',
 'but',
 'can',
 'description',
 'dog',
 'dogs',
 'family',
 'for',
 'forever',
 'foster',
 'from',
 'happy',
 'has',
 'have',
 'he',
 'her',
 'his',
 'home',
 'in',
 'is',
 'just',
 'lbs',
 'love',
 'loves',
 'mix',
 'my',
 'name',
 'not',
 'of',
 'old',
 'on',
 'or',
 'other',
 'our',
 'out',
 'she',
 'shelter',
 'so',
 'sweet',
 't',
 'that',
 'the',
 'to',
 'very',
 'was',
 'we',
 'who',
 'will',
 'with',
 'wonderful',
 'would',
 'year',
 'you']

In [77]:
from sklearn.feature_extraction.text import CountVectorizer


In [81]:
adopted_or_not = np.zeros(1000)

In [98]:
adopted_records

[461, 494, 494, 770]

In [82]:
adopted_or_not[np.array(adopted_records)] += 1


In [115]:
DF = pd.DataFrame(np.array([all_descriptions, adopted_or_not]).T, columns=['descriptions', 'adopted'])

In [116]:
DF.head()

,descriptions,adopted
0,"""description"": {""$t"": ""Meet Blitzen\n\nBlitze...",0.0
1,"""description"": {""$t"": ""Mia (formerly know as ...",0.0
2,"""description"": {""$t"": ""Hi my name is \""Jack\""...",0.0
3,"""description"": {""$t"": ""Blondie belonged to a ...",0.0
4,"""description"": {""$t"": ""Howdy! Caught you look...",0.0


In [112]:
DF.adopted[0:10].values

array(['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
       '0.0'], dtype=object)

In [86]:
vect = CountVectorizer().fit(DF.descriptions)

In [88]:
vect.get_feature_names()[0:10]

['02', '03', '04', '05', '0mths', '0yrs', '10', '100', '100lbs', '1073742072']

In [89]:
X_train_vectorized = vect.transform(DF.descriptions)


In [91]:
from sklearn.linear_model import LogisticRegression


In [117]:
model = LogisticRegression()
model.fit(X_train_vectorized, DF.adopted.values.astype(float).astype(int))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [119]:
predictions = model.predict(X_train_vectorized)


In [120]:
predictions[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [121]:
DF.adopted.value_counts()

0.0    997
1.0      3
Name: adopted, dtype: int64

In [122]:
pd.Series(predictions).value_counts()

0    1000
dtype: int64

In [106]:
DF.adopted.astype(float)[0:10].values.astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [110]:
predictions[0:10].astype(float).astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [123]:
from sklearn.metrics import roc_auc_score

# Predict the transformed test documents
predictions = model.predict(X_train_vectorized)

print('AUC: ', roc_auc_score(DF.adopted.astype(float).values.astype(int), predictions))

AUC:  0.5


### so it goes for now. 